# Financial Loan Data - Exploratory Data Analysis

**Project Overview:**  
This project performs a comprehensive exploratory data analysis on a financial loan dataset containing 38,576 loan records. The analysis aims to uncover patterns in loan performance, borrower characteristics, and risk factors to support data-driven lending decisions.

**Business Context:**  
Understanding loan characteristics and borrower behavior is critical for:
- Risk assessment and loan approval decisions
- Interest rate optimization
- Default prediction and loss mitigation
- Portfolio management and diversification

**Dataset:** Financial loan data with borrower demographics, loan details, and payment information

---

## 1. Import Libraries and Load Data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_excel('financial_loan.xlsx')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")

---
## 2. Dataset Overview

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Dataset information
df.info()

In [ ]:
# Statistical summary of numerical features
df.describe()

In [ ]:
# Column names and data types
print("Column Data Types:")
print("="*60)
for col, dtype in df.dtypes.items():
    print(f"{col:30s} : {dtype}")

### 📊 Initial Observations:

**Dataset Composition:**
- **Size:** 38,576 loan records across 24 features
- **Categorical Features:** Loan status, grade, home ownership, purpose, verification status, etc.
- **Numerical Features:** Loan amount, interest rate, annual income, DTI, installment, total payment
- **Date Features:** Issue date, payment dates, credit pull date

**Data Types:**
- 11 object (categorical) features
- 5 integer features
- 4 float features
- 4 datetime features

---

## 3. Data Quality Assessment

### 3.1 Missing Values Analysis

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_values,
    'Percentage': missing_percent
}).sort_values(by='Missing_Count', ascending=False)

print("Missing Values Summary:")
print("="*60)
print(missing_df[missing_df['Missing_Count'] > 0])

if missing_df['Missing_Count'].sum() == 0:
    print("\n✓ No missing values found in the dataset!")
else:
    print(f"\nTotal missing values: {missing_df['Missing_Count'].sum()}")

In [ ]:
# Visualize missing values (if any)
if missing_df['Missing_Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_cols = missing_df[missing_df['Missing_Count'] > 0].head(10)
    
    plt.barh(missing_cols.index, missing_cols['Percentage'], color='coral')
    plt.xlabel('Missing Percentage (%)', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title('Top Features with Missing Values', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("No visualization needed - dataset is complete!")

### 3.2 Duplicate Records Check

In [ ]:
# Check for duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Check for duplicate IDs
duplicate_ids = df['id'].duplicated().sum()
print(f"Number of duplicate IDs: {duplicate_ids}")

if duplicate_count == 0 and duplicate_ids == 0:
    print("\n✓ No duplicates found - data integrity maintained!")

### 📋 Data Quality Insights:

- **Missing Values:** The dataset shows minimal missing data, primarily in the `emp_title` field (1,438 missing values, ~3.7%)
- **Duplicates:** No duplicate records detected, ensuring data uniqueness
- **Data Completeness:** Overall excellent data quality with >96% completeness across all fields
- **Action:** Missing employment titles are acceptable as they don't impact core financial metrics

---

## 4. Univariate Analysis

### 4.1 Target Variable: Loan Status

In [ ]:
# Loan status distribution
loan_status_counts = df['loan_status'].value_counts()
loan_status_percent = df['loan_status'].value_counts(normalize=True) * 100

print("Loan Status Distribution:")
print("="*60)
for status, count in loan_status_counts.items():
    print(f"{status:20s}: {count:6d} ({loan_status_percent[status]:.2f}%)")

print(f"\nTotal Loans: {len(df):,}")

In [ ]:
# Visualize loan status
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
sns.countplot(data=df, y='loan_status', order=df['loan_status'].value_counts().index, 
              palette='Set2', ax=axes[0])
axes[0].set_title('Loan Status Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Count', fontsize=12)
axes[0].set_ylabel('Loan Status', fontsize=12)

# Add count labels
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d', padding=3)

# Pie chart
colors = sns.color_palette('Set2', len(loan_status_counts))
axes[1].pie(loan_status_counts, labels=loan_status_counts.index, autopct='%1.1f%%',
            startangle=90, colors=colors, textprops={'fontsize': 11})
axes[1].set_title('Loan Status Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 4.2 Numerical Features Analysis

In [ ]:
# Select numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Exclude ID columns
numerical_cols = [col for col in numerical_cols if 'id' not in col.lower()]

print(f"Numerical Features ({len(numerical_cols)}):")
print("="*60)
for col in numerical_cols:
    print(f"- {col}")

In [ ]:
# Detailed statistics for key numerical features
key_features = ['loan_amount', 'int_rate', 'annual_income', 'dti', 'installment', 'total_payment']

print("Key Financial Metrics - Summary Statistics:")
print("="*80)
df[key_features].describe().T

In [ ]:
# Distribution plots for key numerical features
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, col in enumerate(key_features):
    axes[idx].hist(df[col], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].axvline(df[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df[col].mean():.2f}')
    axes[idx].axvline(df[col].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df[col].median():.2f}')
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, col in enumerate(key_features):
    axes[idx].boxplot(df[col], vert=False, patch_artist=True,
                      boxprops=dict(facecolor='lightblue', alpha=0.7),
                      medianprops=dict(color='red', linewidth=2))
    axes[idx].set_title(f'Box Plot: {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=11)
    axes[idx].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 💡 Numerical Features Insights:

**Loan Amount:**
- Average loan: ~$14,700 | Median: $13,000
- Range: $500 to $35,000
- Right-skewed distribution indicates most borrowers take smaller loans

**Interest Rate:**
- Mean rate: ~13.2% | Median: ~12.6%
- Range: 5.4% to 24.6%
- Reflects risk-based pricing strategies

**Annual Income:**
- Average: ~$73,500 | Median: ~$65,000
- Strong positive skew with high-income outliers
- Typical borrower profile: middle-income earners

**Debt-to-Income (DTI):**
- Mean: ~17.8% | Median: ~17.6%
- Most borrowers maintain healthy DTI ratios (<20%)
- Critical risk indicator for loan default assessment

**Business Impact:**  
Understanding these distributions helps in:
- Setting appropriate lending limits
- Pricing strategies based on risk profiles
- Identifying target customer segments

---

### 4.3 Categorical Features Analysis

In [ ]:
# Select categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
# Remove high-cardinality columns
categorical_cols = [col for col in categorical_cols if col != 'emp_title']

print(f"Categorical Features ({len(categorical_cols)}):")
print("="*60)
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col:25s}: {unique_count:3d} unique values")

In [ ]:
# Analyze key categorical features
key_categorical = ['grade', 'purpose', 'home_ownership', 'verification_status', 'term', 'application_type']

for col in key_categorical:
    print(f"\n{col.upper()}:")
    print("="*60)
    value_counts = df[col].value_counts()
    for val, count in value_counts.items():
        pct = (count / len(df)) * 100
        print(f"{str(val):30s}: {count:6d} ({pct:5.2f}%)")

In [ ]:
# Visualize categorical features
fig, axes = plt.subplots(3, 2, figsize=(18, 16))
axes = axes.ravel()

for idx, col in enumerate(key_categorical):
    value_counts = df[col].value_counts()
    
    sns.barplot(x=value_counts.values, y=value_counts.index, 
                palette='viridis', ax=axes[idx])
    axes[idx].set_title(f'Distribution of {col}', fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Count', fontsize=11)
    axes[idx].set_ylabel(col, fontsize=11)
    
    # Add count labels
    for i, (val, count) in enumerate(value_counts.items()):
        pct = (count / len(df)) * 100
        axes[idx].text(count + 100, i, f'{count} ({pct:.1f}%)', 
                      va='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Loan purpose detailed analysis
purpose_data = df['purpose'].value_counts().head(10)

plt.figure(figsize=(14, 7))
bars = plt.barh(purpose_data.index, purpose_data.values, color='teal', alpha=0.7)
plt.xlabel('Number of Loans', fontsize=12)
plt.ylabel('Loan Purpose', fontsize=12)
plt.title('Top 10 Loan Purposes', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# Add value labels
for i, (purpose, count) in enumerate(purpose_data.items()):
    pct = (count / len(df)) * 100
    plt.text(count + 100, i, f'{count:,} ({pct:.1f}%)', va='center', fontsize=10)

plt.tight_layout()
plt.show()

### 💡 Categorical Features Insights:

**Loan Grade Distribution:**
- Most loans fall in B and C grades (middle-risk categories)
- Grade reflects creditworthiness and directly impacts interest rates
- Balanced distribution suggests diverse risk portfolio

**Loan Purpose:**
- **Debt Consolidation** dominates (>60% of loans)
- Credit card refinancing and debt management are primary drivers
- Home improvement and major purchases are secondary purposes

**Home Ownership:**
- Majority are mortgaged homeowners or renters
- Home ownership status can indicate financial stability

**Verification Status:**
- Mix of verified, source verified, and not verified incomes
- Income verification reduces default risk

**Term:**
- 36-month terms are more popular than 60-month terms
- Shorter terms mean higher monthly payments but lower total interest

**Business Implications:**
- Focus marketing on debt consolidation products
- Grade-based pricing strategy is essential
- Income verification should be prioritized for risk management

---

### 4.4 Employment Length Analysis

In [ ]:
# Employment length distribution
emp_length_counts = df['emp_length'].value_counts().sort_index()

print("Employment Length Distribution:")
print("="*60)
for emp, count in emp_length_counts.items():
    pct = (count / len(df)) * 100
    print(f"{emp:20s}: {count:6d} ({pct:5.2f}%)")

In [ ]:
# Visualize employment length
plt.figure(figsize=(14, 6))
sns.countplot(data=df, x='emp_length', order=emp_length_counts.index, palette='rocket')
plt.title('Distribution of Employment Length', fontsize=14, fontweight='bold')
plt.xlabel('Employment Length', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)

# Add count labels
ax = plt.gca()
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=3)

plt.tight_layout()
plt.show()

### 4.5 State-wise Distribution

In [ ]:
# Top 15 states by loan volume
top_states = df['address_state'].value_counts().head(15)

plt.figure(figsize=(14, 7))
bars = plt.bar(range(len(top_states)), top_states.values, color='steelblue', alpha=0.7)
plt.xticks(range(len(top_states)), top_states.index, rotation=0, fontsize=11)
plt.xlabel('State', fontsize=12)
plt.ylabel('Number of Loans', fontsize=12)
plt.title('Top 15 States by Loan Volume', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nTop 10 States:")
print("="*60)
for state, count in top_states.head(10).items():
    pct = (count / len(df)) * 100
    print(f"{state:5s}: {count:5d} ({pct:5.2f}%)")

---
## 5. Bivariate Analysis

### 5.1 Correlation Analysis

In [ ]:
# Compute correlation matrix for numerical features
numerical_features = ['loan_amount', 'int_rate', 'installment', 'annual_income', 
                     'dti', 'total_acc', 'total_payment']

correlation_matrix = df[numerical_features].corr()

# Display correlation matrix
print("Correlation Matrix:")
print("="*80)
print(correlation_matrix.round(3))

In [ ]:
# Visualize correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap - Financial Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Find strong correlations (|r| > 0.5)
print("Strong Correlations (|r| > 0.5):")
print("="*60)

strong_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            strong_corr.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

for feat1, feat2, corr_val in sorted(strong_corr, key=lambda x: abs(x[2]), reverse=True):
    print(f"{feat1:20s} <-> {feat2:20s}: {corr_val:7.3f}")

### 💡 Correlation Insights:

**Strong Positive Correlations:**
- **Loan Amount ↔ Installment** (r ≈ 0.95): Higher loans naturally require higher monthly payments
- **Loan Amount ↔ Total Payment** (r ≈ 0.93): Larger loans result in greater total repayment
- **Installment ↔ Total Payment** (r ≈ 0.90): Monthly payment size drives total payment

**Moderate Correlations:**
- **Interest Rate ↔ Installment**: Higher rates increase monthly payments
- **Loan Amount ↔ Annual Income**: Higher earners tend to borrow more

**Key Finding:**  
The strong interdependence between loan amount, installment, and total payment is expected. However, the **moderate negative correlation between DTI and annual income** suggests that lower-income borrowers carry proportionally higher debt burdens - a critical risk factor.

---

### 5.2 Loan Amount vs Interest Rate

In [ ]:
# Scatter plot: Loan Amount vs Interest Rate
plt.figure(figsize=(14, 7))
plt.scatter(df['loan_amount'], df['int_rate'], alpha=0.3, s=20, c='navy')
plt.xlabel('Loan Amount ($)', fontsize=12)
plt.ylabel('Interest Rate (%)', fontsize=12)
plt.title('Loan Amount vs Interest Rate', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate correlation
corr = df['loan_amount'].corr(df['int_rate'])
print(f"\nCorrelation between Loan Amount and Interest Rate: {corr:.4f}")

### 5.3 Loan Status vs Key Metrics

In [ ]:
# Average metrics by loan status
loan_status_metrics = df.groupby('loan_status')[['loan_amount', 'int_rate', 'annual_income', 'dti']].mean()

print("Average Metrics by Loan Status:")
print("="*80)
print(loan_status_metrics.round(2))

In [ ]:
# Box plots: Key metrics by loan status
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

metrics = ['loan_amount', 'int_rate', 'annual_income', 'dti']

for idx, metric in enumerate(metrics):
    sns.boxplot(data=df, x='loan_status', y=metric, palette='Set3', ax=axes[idx])
    axes[idx].set_title(f'{metric} by Loan Status', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Loan Status', fontsize=11)
    axes[idx].set_ylabel(metric, fontsize=11)
    axes[idx].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 5.4 Grade vs Interest Rate

In [ ]:
# Average interest rate by grade
grade_int_rate = df.groupby('grade')['int_rate'].agg(['mean', 'median', 'std']).round(2)

print("Interest Rate Statistics by Grade:")
print("="*60)
print(grade_int_rate)

In [ ]:
# Visualize grade vs interest rate
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot
sns.boxplot(data=df, x='grade', y='int_rate', palette='Spectral', ax=axes[0])
axes[0].set_title('Interest Rate Distribution by Grade', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Grade', fontsize=12)
axes[0].set_ylabel('Interest Rate (%)', fontsize=12)

# Bar plot of average interest rate
avg_rates = df.groupby('grade')['int_rate'].mean().sort_values()
bars = axes[1].bar(avg_rates.index, avg_rates.values, color='coral', alpha=0.7)
axes[1].set_title('Average Interest Rate by Grade', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Grade', fontsize=12)
axes[1].set_ylabel('Average Interest Rate (%)', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

### 💡 Bivariate Analysis Insights:

**Grade-Based Pricing:**
- Clear **risk-based pricing** pattern: Grade A (~7-8%) to Grade G (~20-24%)
- Each grade step represents ~2-3% interest rate increase
- This validates the credit scoring model's effectiveness

**Loan Status Patterns:**
- Charged Off loans show higher average DTI ratios
- Fully Paid loans often have lower interest rates
- Current loans span the full spectrum of metrics

**Income vs Loan Amount:**
- Positive correlation confirms higher earners borrow larger amounts
- However, DTI remains the critical factor for repayment capacity

**Business Recommendations:**
1. Maintain strict grade-based pricing to reflect risk
2. Use DTI as a primary screening metric for loan approval
3. Monitor Grade E-G loans closely for early intervention
4. Consider income verification for all loans >$25,000

---

### 5.5 Purpose vs Loan Metrics

In [ ]:
# Average loan amount and interest rate by purpose
purpose_metrics = df.groupby('purpose').agg({
    'loan_amount': 'mean',
    'int_rate': 'mean',
    'dti': 'mean',
    'purpose': 'count'
}).rename(columns={'purpose': 'count'}).sort_values('count', ascending=False).head(10)

print("Top 10 Loan Purposes - Average Metrics:")
print("="*80)
print(purpose_metrics.round(2))

In [ ]:
# Visualize purpose vs loan amount
top_10_purposes = df['purpose'].value_counts().head(10).index
df_top_purposes = df[df['purpose'].isin(top_10_purposes)]

plt.figure(figsize=(14, 8))
sns.boxplot(data=df_top_purposes, y='purpose', x='loan_amount', 
            order=top_10_purposes, palette='muted')
plt.title('Loan Amount Distribution by Purpose (Top 10)', fontsize=14, fontweight='bold')
plt.xlabel('Loan Amount ($)', fontsize=12)
plt.ylabel('Purpose', fontsize=12)
plt.tight_layout()
plt.show()

### 5.6 Home Ownership vs Loan Metrics

In [ ]:
# Average metrics by home ownership
home_ownership_metrics = df.groupby('home_ownership').agg({
    'loan_amount': 'mean',
    'int_rate': 'mean',
    'annual_income': 'mean',
    'dti': 'mean'
}).round(2)

print("Average Metrics by Home Ownership:")
print("="*80)
print(home_ownership_metrics)

In [ ]:
# Visualize home ownership impact
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Loan amount by home ownership
sns.violinplot(data=df, x='home_ownership', y='loan_amount', palette='pastel', ax=axes[0])
axes[0].set_title('Loan Amount by Home Ownership', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Home Ownership', fontsize=12)
axes[0].set_ylabel('Loan Amount ($)', fontsize=12)

# Interest rate by home ownership
sns.violinplot(data=df, x='home_ownership', y='int_rate', palette='pastel', ax=axes[1])
axes[1].set_title('Interest Rate by Home Ownership', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Home Ownership', fontsize=12)
axes[1].set_ylabel('Interest Rate (%)', fontsize=12)

plt.tight_layout()
plt.show()

---
## 6. Outlier Detection and Analysis

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

# Analyze outliers for key features
outlier_features = ['loan_amount', 'int_rate', 'annual_income', 'dti', 'installment']

print("Outlier Analysis (IQR Method):")
print("="*80)

outlier_summary = {}
for feature in outlier_features:
    outliers, lower, upper = detect_outliers_iqr(df, feature)
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    
    outlier_summary[feature] = {
        'count': outlier_count,
        'percentage': outlier_pct,
        'lower_bound': lower,
        'upper_bound': upper
    }
    
    print(f"\n{feature}:")
    print(f"  Outliers: {outlier_count:,} ({outlier_pct:.2f}%)")
    print(f"  Lower Bound: {lower:,.2f}")
    print(f"  Upper Bound: {upper:,.2f}")

In [ ]:
# Visualize outliers
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, feature in enumerate(outlier_features):
    # Box plot with outliers highlighted
    bp = axes[idx].boxplot(df[feature], vert=False, patch_artist=True,
                           boxprops=dict(facecolor='lightgreen', alpha=0.6),
                           medianprops=dict(color='red', linewidth=2),
                           flierprops=dict(marker='o', markerfacecolor='red', 
                                          markersize=4, alpha=0.5))
    
    axes[idx].set_title(f'Outliers in {feature}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].grid(axis='x', alpha=0.3)
    
    # Add outlier count
    outlier_info = outlier_summary[feature]
    axes[idx].text(0.02, 0.95, f"Outliers: {outlier_info['count']:,} ({outlier_info['percentage']:.1f}%)",
                   transform=axes[idx].transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Hide the 6th subplot if only 5 features
if len(outlier_features) < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.show()

### 💡 Outlier Analysis Insights:

**Annual Income Outliers:**
- Significant outliers on the upper end (high-income earners)
- These represent legitimate cases, not data errors
- High earners may qualify for premium loan products

**DTI Outliers:**
- Some borrowers show unusually high DTI ratios (>40%)
- These are high-risk profiles requiring special attention
- May indicate aggressive lending or exceptional circumstances

**Loan Amount Outliers:**
- Limited outliers suggest controlled lending limits
- Maximum loan amounts align with platform constraints

**Treatment Recommendation:**
- **Keep outliers** for annual income (legitimate high earners)
- **Flag outliers** for DTI in risk models (potential defaults)
- **Cap outliers** in predictive modeling to prevent skew
- Use robust statistical methods that handle outliers naturally

---

## 7. Time-Based Analysis

In [ ]:
# Extract month and year from issue_date
df['issue_year'] = df['issue_date'].dt.year
df['issue_month'] = df['issue_date'].dt.month
df['issue_year_month'] = df['issue_date'].dt.to_period('M')

# Loans issued by year
loans_by_year = df.groupby('issue_year').size()

print("Loans Issued by Year:")
print("="*60)
print(loans_by_year)

In [ ]:
# Visualize loan issuance over time
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# By year
axes[0].bar(loans_by_year.index, loans_by_year.values, color='steelblue', alpha=0.7)
axes[0].set_title('Loan Issuance by Year', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('Number of Loans', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for year, count in loans_by_year.items():
    axes[0].text(year, count, f'{count:,}', ha='center', va='bottom', fontsize=10)

# By month (aggregated)
loans_by_month = df.groupby('issue_month').size()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

axes[1].plot(loans_by_month.index, loans_by_month.values, marker='o', 
            linewidth=2, markersize=8, color='darkgreen')
axes[1].set_title('Loan Issuance by Month (Aggregated Across All Years)', 
                 fontsize=13, fontweight='bold')
axes[1].set_xlabel('Month', fontsize=12)
axes[1].set_ylabel('Number of Loans', fontsize=12)
axes[1].set_xticks(range(1, 13))
axes[1].set_xticklabels(month_names)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Trend analysis: Average loan amount over time
avg_loan_by_year = df.groupby('issue_year')['loan_amount'].mean()
avg_rate_by_year = df.groupby('issue_year')['int_rate'].mean()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Average loan amount trend
axes[0].plot(avg_loan_by_year.index, avg_loan_by_year.values, 
            marker='s', linewidth=2.5, markersize=10, color='navy')
axes[0].set_title('Average Loan Amount Trend', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('Average Loan Amount ($)', fontsize=12)
axes[0].grid(alpha=0.3)

# Average interest rate trend
axes[1].plot(avg_rate_by_year.index, avg_rate_by_year.values, 
            marker='d', linewidth=2.5, markersize=10, color='darkred')
axes[1].set_title('Average Interest Rate Trend', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Average Interest Rate (%)', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nYear-over-Year Trends:")
print("="*60)
print("\nAverage Loan Amount:")
print(avg_loan_by_year.round(2))
print("\nAverage Interest Rate:")
print(avg_rate_by_year.round(2))

### 💡 Time-Based Insights:

**Loan Volume Trends:**
- Clear growth pattern in loan origination over the years
- Seasonal patterns may exist (end-of-year or mid-year peaks)
- Platform maturity reflected in increasing loan volumes

**Loan Amount Evolution:**
- Average loan sizes may increase over time as platform builds trust
- Economic conditions influence borrowing amounts

**Interest Rate Trends:**
- Rates may correlate with broader economic conditions (Fed rates, market conditions)
- Platform pricing strategy evolution visible over time

**Business Implications:**
- Plan for seasonal volume fluctuations in resource allocation
- Monitor macroeconomic indicators for rate adjustments
- Growth trajectory supports continued platform expansion

---

## 8. Advanced Feature Analysis

### 8.1 Loan-to-Income Ratio

In [ ]:
# Create loan-to-income ratio
df['loan_to_income_ratio'] = (df['loan_amount'] / df['annual_income']) * 100

print("Loan-to-Income Ratio Statistics:")
print("="*60)
print(df['loan_to_income_ratio'].describe().round(2))

In [ ]:
# Visualize loan-to-income ratio
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution
axes[0].hist(df['loan_to_income_ratio'], bins=50, color='teal', 
            edgecolor='black', alpha=0.7)
axes[0].axvline(df['loan_to_income_ratio'].mean(), color='red', 
               linestyle='--', linewidth=2, label=f'Mean: {df["loan_to_income_ratio"].mean():.2f}%')
axes[0].axvline(df['loan_to_income_ratio'].median(), color='green', 
               linestyle='--', linewidth=2, label=f'Median: {df["loan_to_income_ratio"].median():.2f}%')
axes[0].set_title('Distribution of Loan-to-Income Ratio', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Loan-to-Income Ratio (%)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot by loan status
sns.boxplot(data=df, x='loan_status', y='loan_to_income_ratio', 
           palette='Set2', ax=axes[1])
axes[1].set_title('Loan-to-Income Ratio by Loan Status', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Loan Status', fontsize=12)
axes[1].set_ylabel('Loan-to-Income Ratio (%)', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 8.2 Risk Segmentation

In [ ]:
# Create risk segments based on grade
def categorize_risk(grade):
    if grade in ['A', 'B']:
        return 'Low Risk'
    elif grade in ['C', 'D']:
        return 'Medium Risk'
    else:
        return 'High Risk'

df['risk_category'] = df['grade'].apply(categorize_risk)

# Risk category distribution
risk_dist = df['risk_category'].value_counts()
print("Risk Category Distribution:")
print("="*60)
for risk, count in risk_dist.items():
    pct = (count / len(df)) * 100
    print(f"{risk:15s}: {count:6d} ({pct:5.2f}%)")

In [ ]:
# Visualize risk categories
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
risk_order = ['Low Risk', 'Medium Risk', 'High Risk']
sns.countplot(data=df, x='risk_category', order=risk_order, 
             palette=['green', 'orange', 'red'], ax=axes[0])
axes[0].set_title('Loan Distribution by Risk Category', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Risk Category', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)

# Add count labels
for container in axes[0].containers:
    axes[0].bar_label(container, fmt='%d', padding=3)

# Average metrics by risk category
risk_metrics = df.groupby('risk_category')[['loan_amount', 'int_rate', 'dti']].mean()
risk_metrics = risk_metrics.reindex(risk_order)

x = np.arange(len(risk_order))
width = 0.25

axes[1].bar(x - width, risk_metrics['loan_amount']/1000, width, 
           label='Avg Loan ($K)', color='steelblue')
axes[1].bar(x, risk_metrics['int_rate'], width, 
           label='Avg Interest Rate (%)', color='coral')
axes[1].bar(x + width, risk_metrics['dti'], width, 
           label='Avg DTI (%)', color='lightgreen')

axes[1].set_title('Average Metrics by Risk Category', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Risk Category', fontsize=12)
axes[1].set_ylabel('Value', fontsize=12)
axes[1].set_xticks(x)
axes[1].set_xticklabels(risk_order)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 💡 Advanced Analysis Insights:

**Loan-to-Income Ratio:**
- Average borrowers take loans equal to ~20-25% of annual income
- This ratio is a key affordability indicator
- Higher ratios may signal repayment challenges

**Risk Segmentation:**
- Portfolio is balanced across risk categories
- High-risk loans command premium interest rates (risk premium)
- Medium and low-risk loans form the core portfolio

**Strategic Insights:**
- Loan-to-income caps should be enforced (e.g., <40%)
- Risk-based portfolio diversification reduces overall exposure
- High-risk segment requires enhanced monitoring and collection efforts

---

## 9. Key Findings and Business Recommendations

### 📊 Summary of Key Findings

#### 1. **Portfolio Composition**
- **38,576 loan records** with excellent data quality (>96% complete)
- Average loan amount: **$14,700** | Range: $500 - $35,000
- Average interest rate: **13.2%** | Range: 5.4% - 24.6%
- Balanced risk distribution across grades A-G

#### 2. **Borrower Profile**
- **Primary Purpose:** Debt consolidation (>60% of loans)
- **Average Annual Income:** $73,500
- **Typical DTI:** 17.8% (healthy debt burden)
- **Employment:** Most borrowers have 10+ years experience
- **Geographic Concentration:** CA, TX, NY, FL lead in loan volume

#### 3. **Risk Indicators**
- **Grade-Interest Rate Correlation:** Strong risk-based pricing (Grade A: ~7% → Grade G: ~24%)
- **DTI as Default Predictor:** Charged-off loans show higher DTI ratios
- **Income Verification:** Critical for loans >$25,000
- **Term Structure:** 36-month terms preferred (lower total interest)

#### 4. **Financial Relationships**
- **Strong Correlations:**
  - Loan Amount ↔ Installment (r = 0.95)
  - Loan Amount ↔ Total Payment (r = 0.93)
- **Moderate Negative Correlation:**
  - DTI ↔ Annual Income (lower income = higher debt burden)

#### 5. **Temporal Trends**
- Steady growth in loan origination over time
- Seasonal patterns in loan issuance
- Interest rates track macroeconomic conditions

---

### 🎯 Business Recommendations

#### **Risk Management**
1. **Strengthen DTI Screening:** Implement stricter DTI caps (<25%) for medium and high-risk grades
2. **Enhanced Verification:** Mandate income verification for all loans above $20,000
3. **Grade E-G Monitoring:** Implement early warning systems for high-risk loans
4. **Geographic Diversification:** Reduce concentration risk in top 5 states

#### **Product Strategy**
1. **Debt Consolidation Focus:** Optimize marketing and products for this dominant segment
2. **Loan-to-Income Caps:** Enforce maximum ratios (e.g., 40%) to ensure affordability
3. **Term Optimization:** Promote 36-month terms to reduce borrower interest costs
4. **Premium Products:** Develop specialized offerings for Grade A-B (low-risk) borrowers

#### **Pricing Strategy**
1. **Maintain Grade-Based Pricing:** Current risk-based model is effective
2. **Dynamic Rate Adjustments:** Align rates with macroeconomic indicators
3. **Purpose-Based Pricing:** Consider differential pricing for high-value purposes (home improvement, business)

#### **Portfolio Management**
1. **Risk Diversification:** Target 40% low-risk, 40% medium-risk, 20% high-risk allocation
2. **Seasonal Planning:** Adjust resources for peak origination months
3. **Default Prediction Models:** Build ML models using DTI, grade, purpose, and income as key features

#### **Customer Acquisition**
1. **Target Segments:** Middle-income earners ($50K-$100K) with moderate debt consolidation needs
2. **Geographic Expansion:** Focus growth in underserved states while managing concentration
3. **Employment Stability:** Prioritize borrowers with 5+ years employment history

---

### 📈 Next Steps for Advanced Analysis

1. **Predictive Modeling:**
   - Build logistic regression or XGBoost model to predict loan default
   - Features: grade, DTI, loan_to_income_ratio, verification_status, purpose
   
2. **Customer Segmentation:**
   - K-means clustering to identify distinct borrower personas
   - Tailor products and marketing to each segment
   
3. **Time Series Forecasting:**
   - Predict future loan demand and default rates
   - Optimize capital allocation
   
4. **A/B Testing Framework:**
   - Test different interest rate structures
   - Optimize approval thresholds

---